### IMPORTS

In [1]:
import json
import time
import hashlib
import base64

In [2]:
from web3 import Web3

In [3]:
print('[X] Booting up...')

[X] Booting up...


### LOAD DATA FROM EXTERNAL FILES

In [4]:
with open('config/settings.json') as json_file:
    settings = json.load(json_file)

In [5]:
with open('config/identifier.json') as json_file:
    device_info = json.load(json_file)

In [6]:
with open('config/latest.json') as json_file:
    latest = json.load(json_file)

### SERIALIZE DEVICE

In [7]:
def hash_id(data):
    
    # REMOVE WHITESPACES
    to_string = json.dumps(data, sort_keys=False, indent=2)
    
    # ENCODE THE STRING WITH UTF8
    encoded = to_string.encode("utf-8")
    
    # HASH ENCODED DATA
    hashed = hashlib.sha256(encoded).hexdigest()
    
    return hashed

In [8]:
class create_device():
    def __init__(self):
        self.name = device_info['Name']
        self.hash = hash_id(device_info)
    
    # LOCATE & SET DEVICE CONTRACT ADDRESS
    def set_contract(self, _contract):
        self.contract = _contract
    
    # REDIRECT TO SMART CONTRACT INTERFACE
    def read(self, details):
        return self.contract.read(details)
    
    # REDIRECT TO SMART CONTRACT INTERFACE
    def write(self, details):
        return self.contract.write(details)
    
    # REDIRECT TO SMART CONTRACT INTERFACE
    def event(self, name):
        return self.contract.event(name)

In [9]:
device = create_device()

### CONNECT TO WHISPER

In [10]:
whisper = Web3(Web3.WebsocketProvider('ws://' + settings['gateways']['whisper']['host'] + ':' + str(settings['gateways']['whisper']['port'])))

In [11]:
if whisper.isConnected():
    print('[X] Connected with whisper gateway!')
else:
    print('[ ] Could not connect to whisper gateway!')
    exit

[X] Connected with whisper gateway!


### CONNECT TO BLOCKCHAIN

In [12]:
web3 = Web3(Web3.WebsocketProvider('ws://' + settings['gateways']['blockchain']['host'] + ':' + settings['gateways']['blockchain']['port']))

### CREATE CONTRACT OUTLINE

In [13]:
class contract:
    
    # ON LOAD..
    def __init__(self, block):
        
        # CONSTRUCT USABLE CONTRACT
        self.contract = web3.eth.contract(
            address = block['address'],
            abi = block['abi']
        )
        
        # SET ADDRESS REFERENCE
        self.address = block['address']
    
    # READ FROM CONTRACT
    def read(self, details):
        
        # WITH PARAMS
        if ('params' in details):
            return self.contract.functions[details['func']](details['params']).call()
        
        # WITHOUT PARAMS
        else:
            return self.contract.functions[details]().call()
    
    # EVENT FILTER
    def event(self, name):
        return self.contract.events[name].createFilter(fromBlock="latest")

### SERIALIZE DEVICE MANAGER & DEVICE CONTRACT

In [14]:
device_manager = contract(latest['devicemanager'])

In [15]:
print('[X] Device manager set!')

[X] Device manager set!


In [16]:
temp_contract = contract({
    'address': device_manager.read({
        'func': 'fetch_device',
        'params': device.hash
    }),
    'abi': latest['device']['abi']
})

In [17]:
device.set_contract(temp_contract)

In [18]:
print('[X] Device contract set!')

[X] Device contract set!


### FETCH WHISPER API

In [19]:
shh = whisper.geth.shh

### GENERATE WHISPER KEYPAIR

In [20]:
whisper_id = shh.newKeyPair()

In [21]:
print('[X] Generated keypair!')

[X] Generated keypair!


### SUBSCRIBE TO UPDATE EVENTS

In [22]:
print('[X] Listening for messages...\n')

[X] Listening for messages...



### WHISPER & SMART CONTRACT EVENTS

In [23]:
message_event = shh.newMessageFilter({
    'topic': web3.toHex(text=settings['whisper']['topic']['name']),
    'symKeyID': settings['whisper']['topic']['key']
})

In [24]:
config_event = device.event('config')

### MODIFY DISCOVERY CONFIG

In [25]:
def parse_config(compressed):
    
    # EXPAND TO STRING
    to_bytes = base64.b64decode(compressed)
    
    # PARSE AS JSON & RETURN
    return json.loads(to_bytes)

### FETCH CONFIG FROM SMART CONTRACT

In [26]:
compressed = device.read('tags')

In [27]:
discovery_config = parse_config(compressed)

### CONTENT DISCOVERY PARSER

In [28]:
def content_discovery(event):

    # CONVERT MESSAGE PARAMS
    author = web3.toHex(event['sig'])
    nickname = author[0:4] + '...' + author[-4:]
    message = web3.toText(event['payload'])

    # CHECK THE FIRST WORD FOR TRIGGER
    first = message.split(' ')[0].lower()

    # IF A REQUEST IS FOUND
    if (first == '!request'):

        # SIGNAL REQUEST
        print('New request found!')

        # NUKE GARBAGE & SPLIT OUT INDIVIDUAL PARAMS
        checks = message[8:].replace(' ', '').split('&')

        # CHECK RESULTS
        results = []

        # LOOP THROUGH
        for pair in checks:

            # DECONSTRUCT KEY/VALUE
            key, value = pair.split('=')

            # MAKE SURE THE KEY EXISTS
            if (key in discovery_config):

                # CHECK & PUSH PARAM COMPARISON RESULT
                result = discovery_config[key] == value
                results.append(result)

            # OTHERWISE, DEFAULT TO BAD RESULT
            else:
                results.append(False)

        # IF ALL THE CHECKS MATCH
        if (results.count(False) == 0):

            # RESPONSE TEXT
            response = '@' + nickname + ' - I am available at ' + device_id

            # SLEEP FOR 2 SECONDS
            time.sleep(2)

            # RESPOND TO REQUEST
            shh.post({
                'symKeyID': settings['whisper']['topic']['key'],
                'payload': web3.toHex(text=response),
                'topic': web3.toHex(text=settings['whisper']['topic']['name']),
                'sig': whisper_id,
                'powTarget': 2.5,
                'powTime': 2
            })

### SYSTEM LOOP

In [ ]:
while(True):
    
    # CONFIG CHANGE EVENT
    for event in config_event.get_new_entries():
        
        # PRINT MSG
        print('Config update detected!')
        
        # DECONSTRUCT THE COMPRESSED STRING
        compressed = event['args']['data']
        
        # DECODE & UPDATE THE CONFIG
        discovery_config = parse_config(compressed)
    
    # ON WHISPER MESSAGE
    for event in shh.getMessages(message_event):
        
        # PRINT MSG
        print('Whisper event triggered!')
        
        # PARSE THE REQUEST
        content_discovery(event)